In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from datasets import load_dataset



c:\Users\franc\Documents\Data_Science\Anaconda_Navigator\envs\law_chat\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load your data
data_dir = 'C:\\Users\\franc\\Documents\\GitHub\\Car_Detection_Chat\\testando'

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)


Found 0 files belonging to 0 classes.
Using 0 files for training.


ValueError: No images found in directory C:\Users\franc\Documents\GitHub\Car_Detection_Chat\testando. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [24]:
# CHECK ENCODING
def check_utf8_encoding(data_dir):
    for root, dirs, files in os.walk(data_dir):
        # Check directory names
        for dir_name in dirs:
            try:
                dir_name.encode('utf-8').decode('utf-8')
            except UnicodeDecodeError:
                print(f"Non-UTF-8 directory name detected: {os.path.join(root, dir_name)}")
        
        # Check file names
        for file_name in files:
            try:
                file_name.encode('utf-8').decode('utf-8')
            except UnicodeDecodeError:
                print(f"Non-UTF-8 file name detected: {os.path.join(root, file_name)}")

data_dir = 'C:\\Users\\franc\\Documents\\GitHub\\Car_Detection_Chat\\images'
check_utf8_encoding(data_dir)

In [ ]:
Cars = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))


In [22]:
import os
from PIL import Image

for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except Exception as e:
                print(f"Problem with file: {file_path}. Error: {e}")
